## youtube Crawling

In [7]:
# !pip install selenium
# !pip install webdriver-manager

In [1]:
import sys
import time 
from selenium import webdriver
from selenium.webdriver import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# 크롬 웹드라이버 설치 및 설정
Chrome_options = webdriver.ChromeOptions()

In [2]:
# 웹페이지에서 우리 컴이 봇으로 인식하지 못하게 하기 위한 작업 
# 인터넷에 user agent string 치고 각자 컴퓨터에 맞게 설정

user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
Chrome_options.add_argument('user-agent=' + user_agent)

In [6]:
Chrome_options.add_experimental_option('detach', True)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=Chrome_options)
driver.maximize_window()

In [9]:
keyword = 'me+at+the+zoo'
count = 10 

driver.get('https://www.youtube.com/results?search_query='+ keyword)
driver.implicitly_wait(5)
time.sleep(1)

# 영상 선택
driver.find_element(By.XPATH, '//*[@id="thumbnail"]/yt-image/img').click()
# 재생 중인 영상 중지
driver.find_element(By.XPATH, '//*[@id="movie_player"]/div[30]/div[2]/div[1]/button').click()

# 끝까지 스크롤 내리기
driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.END)
time.sleep(1)

# 필요한 댓글 확보를 위한 while문
while True:

    # 스크롤 현재 높이 저장
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    print('new_height:',new_height)
    time.sleep(1)
    # while 문 멈출지 체크 

    if count == -1:  # 페이지 끝까지 while문 돌리기
        driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.END)
        time.sleep(1)

        # 스크롤 후 현재 높이 저장
        last_height = driver.execute_script("return document.documentElement.scrollHeight")
        print('last_height:',last_height)
        time.sleep(1)

        # 페이지 끝까지 스크롤 했는지 판단
        if new_height == last_height:   # if(이전 스크롤 높이 = 현재 스크롤 높이 ) 마지막 페이지라 판단. while 문 탈출
            break
    else:
        #  페이지 읽기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        time.sleep(1)

        # 댓글 구역 추출하기
        comments_sections = soup.find_all('ytd-comment-thread-renderer', class_='ytd-item-section-renderer')

        if len(comments_sections) < count:  # 수집하려는 개수보다 확보된 댓글이 많다면
            # 페이지 끝까지 스크롤 내리기 
            driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.END)
            time.sleep(1)

            # 스크롤 후 현재 높이 저장
            last_height = driver.execute_script("return document.documentElement.scrollHeight")
            print('last_height:',last_height)
            time.sleep(1)

            # 페이지 끝까지 스크롤 했는지 판단
            if new_height == last_height:   # if(이전 스크롤 높이 = 현재 스크롤 높이 ) 마지막 페이지라 판단. while 문 탈출
                break
        else:
            print(f'{count}만큼 댓글 확보 완료')
            del comments_sections[count:]
            break
        
print('페이지 스크롤 완료')


new_height: 2197
last_height: 4177
new_height: 4237
last_height: 6050
new_height: 6277
last_height: 8325
new_height: 8325
50만큼 댓글 확보 완료
페이지 스크롤 완료


In [10]:
comments_sections

[<ytd-comment-thread-renderer class="style-scope ytd-item-section-renderer"><!--css-build:shady--><!--css-build:shady-->
 <ytd-comment-renderer class="style-scope ytd-comment-thread-renderer" comment-style="unknown" id="comment" style="--ytd-comment-paid-background-color: initial;"><!--css-build:shady--><!--css-build:shady--><div class="style-scope ytd-comment-renderer" id="paid-comment-background"></div>
 <div class="style-scope ytd-comment-renderer" hidden="" id="linked-comment-badge"></div>
 <div class="style-scope ytd-comment-renderer" id="body">
 <div class="style-scope ytd-comment-renderer" id="author-thumbnail">
 <a class="yt-simple-endpoint style-scope ytd-comment-renderer" href="/channel/UCC5NfQ6Mf0dq_eEwv4P_hWA">
 <yt-img-shadow class="style-scope ytd-comment-renderer no-transition empty" fit="" height="40" style="background-color: transparent;" width="40"><!--css-build:shady--><!--css-build:shady--><img alt="" class="style-scope yt-img-shadow" draggable="false" height="40" i

In [11]:
username = []
usercomment = []
for item in comments_sections:
    try:
        a = item.find('yt-formatted-string', id='text').get_text()
        b = item.find('yt-formatted-string', id='content-text').get_text()
    except:
        a = item.find('a', id='author-text').get_text().strip()
        b = item.find('yt-formatted-string', id='content-text').get_text()
    username.append(a)
    usercomment.append(b)

# 확인
print(f'count: {len(usercomment)}')
print('{username: usercomment} is')
for i, j in zip(username, usercomment):
    print(f'{{{i}: {j}}}')

count: 50
{username: usercomment} is
{San Diego Zoo: We're so honored that the first ever YouTube video was filmed here!}
{Alan Walker: Just got this piece of art recommended, never gets old!}
{riversongstar ★: 18 years, and this has become one of the biggest video platforms. Thanks, Jawed.}
{lord raiden: It may be a 19 second video but it was the first of almost millions of videos but it will always be the one that started a legend of a group of friends who if they saw what 18 years later raised something that would not only become a legend but also created a pillar on the internet}
{PRO CAT YT: 18 years later and it is still as fresh as like it released yesterday . What a masterpiece}
{Ruvyzvat: I feel like the only proper dedication we could provide is to make this the most viewed video on the entire platform. Just the thought of those billions of videos all fighting for top spot and it was taken on a whim by the very first video that was only a test would be hilarious}
{Satori Grap